In [1]:
from pathlib import Path  # no installation needed
import os  # no installation needed
import sys  # no installation needed
import calendar  # no installation needed
import pandas as pd  # already in env — no new install

DATA_ROOT = Path(r"C:\Users\quantbase\Desktop\marketdata")
REPO_ROOT = Path(r"C:\Users\quantbase\Desktop\sydata")
MANIFEST  = DATA_ROOT / "meta" / "symbols.yml"
SRC       = REPO_ROOT / "src"

sys.path.insert(0, str(SRC))
os.chdir(str(REPO_ROOT))

from sydata.io.symbols import load_manifest, load_basket  # project-local

spec = load_manifest(MANIFEST)
symbols = load_basket(spec, "core_major")

INTERVAL = "15m"
YEAR = 2025
MONTHS = list(range(1, 13))  # or [12] for testing


In [2]:
# loaders plus month slicing

def month_bounds_utc_ms(year: int, month: int) -> tuple[int, int]:
    days = calendar.monthrange(year, month)[1]
    start = pd.Timestamp(year, month, 1, tz="UTC")
    end = start + pd.Timedelta(days=days)
    return int(start.value // 1_000_000), int(end.value // 1_000_000)

def load_spot_year(symbol: str, interval: str, start_ms: int, end_ms: int) -> pd.DataFrame:
    p = DATA_ROOT / "raw" / "binance" / "klines" / f"symbol={symbol}" / f"interval={interval}" / f"part-{start_ms}-{end_ms}.parquet"
    df = pd.read_parquet(p)
    if "ts" not in df.columns:
        df["ts"] = pd.to_datetime(df["open_time"], unit="ms", utc=True)
    return df

def load_um_month(dataset: str, symbol: str, interval: str, year: int, month: int) -> pd.DataFrame:
    p = DATA_ROOT / "raw" / "binance" / dataset / f"symbol={symbol}" / f"interval={interval}" / f"part-{year:04d}-{month:02d}.parquet"
    df = pd.read_parquet(p)
    if "ts" not in df.columns:
        df["ts"] = pd.to_datetime(df["open_time"], unit="ms", utc=True)
    return df

def load_funding_month(symbol: str, year: int, month: int) -> pd.DataFrame:
    p = DATA_ROOT / "raw" / "binance" / "um_funding_rate" / f"symbol={symbol}" / f"part-{year:04d}-{month:02d}.parquet"
    df = pd.read_parquet(p)
    if "ts" not in df.columns and "funding_time" in df.columns:
        df["ts"] = pd.to_datetime(df["funding_time"], unit="ms", utc=True)
    return df


In [3]:
# master builder


def make_master_slice(spot_df, mark_df, index_df, prem_df, fund_df) -> pd.DataFrame:
    s = spot_df[["open_time","ts","close","volume","quote_volume","trades"]].rename(columns={"close":"spot_close"})
    m = mark_df[["open_time","close"]].rename(columns={"close":"mark_close"})
    i = index_df[["open_time","close"]].rename(columns={"close":"index_close"})
    p = prem_df[["open_time","close"]].rename(columns={"close":"premium_close"})
    f = fund_df[["ts","funding_rate","funding_interval_hours"]].sort_values("ts")

    master = s.merge(m, on="open_time", how="left").merge(i, on="open_time", how="left").merge(p, on="open_time", how="left")
    master["basis_mark_vs_spot"]  = (master["mark_close"]  / master["spot_close"]) - 1.0
    master["basis_index_vs_spot"] = (master["index_close"] / master["spot_close"]) - 1.0

    master = pd.merge_asof(
        master.sort_values("ts"),
        f.sort_values("ts"),
        on="ts",
        direction="backward",
        allow_exact_matches=True,
    )

    cols = [
        "ts","open_time","spot_close","mark_close","index_close","premium_close",
        "basis_mark_vs_spot","basis_index_vs_spot",
        "funding_rate","funding_interval_hours",
        "volume","quote_volume","trades",
    ]
    cols = [c for c in cols if c in master.columns] + [c for c in master.columns if c not in cols]
    return master[cols].sort_values("ts").reset_index(drop=True)

def audit_master(master: pd.DataFrame) -> dict:
    return {
        "rows": len(master),
        "mark_missing_frac": float(master["mark_close"].isna().mean()),
        "index_missing_frac": float(master["index_close"].isna().mean()),
        "prem_missing_frac": float(master["premium_close"].isna().mean()),
        "funding_missing_frac": float(master["funding_rate"].isna().mean()),
        "min_ts": str(master["ts"].min()),
        "max_ts": str(master["ts"].max()),
    }


In [5]:
# loop symbol over time

# your spot files for YEAR are full-range; define them once per symbol
YEAR_START_MS, YEAR_END_MS = month_bounds_utc_ms(YEAR, 1)[0], month_bounds_utc_ms(YEAR, 12)[1]

OUT_BASE = DATA_ROOT / "norm" / "master" / f"interval={INTERVAL}" / f"year={YEAR}"
OUT_BASE.mkdir(parents=True, exist_ok=True)

reports = {}

for sym in symbols:
    spot_year = load_spot_year(sym, INTERVAL, YEAR_START_MS, YEAR_END_MS)

    for mo in MONTHS:
        m0, m1 = month_bounds_utc_ms(YEAR, mo)

        # critical: slice spot to the month you are building
        spot_m = spot_year[(spot_year["open_time"] >= m0) & (spot_year["open_time"] < m1)].copy()
        if spot_m.empty:
            reports[(sym, mo)] = {"ok": False, "reason": "spot_empty"}
            continue

        # load same month UM datasets
        mark_m  = load_um_month("um_mark_price_klines", sym, INTERVAL, YEAR, mo)
        index_m = load_um_month("um_index_price_klines", sym, INTERVAL, YEAR, mo)
        prem_m  = load_um_month("um_premium_index_klines", sym, INTERVAL, YEAR, mo)
        fund_m  = load_funding_month(sym, YEAR, mo)

        master = make_master_slice(spot_m, mark_m, index_m, prem_m, fund_m)

        out_path = OUT_BASE / f"month={mo:02d}" / f"symbol={sym}" / f"part-{YEAR:04d}-{mo:02d}.parquet"
        out_path.parent.mkdir(parents=True, exist_ok=True)
        master.to_parquet(out_path, index=False)

        reports[(sym, mo)] = {"ok": True, "out": str(out_path), **audit_master(master)}

reports


{('BTC-USDT', 1): {'ok': True,
  'out': 'C:\\Users\\quantbase\\Desktop\\marketdata\\norm\\master\\interval=15m\\year=2025\\month=01\\symbol=BTC-USDT\\part-2025-01.parquet',
  'rows': 2976,
  'mark_missing_frac': 0.0,
  'index_missing_frac': 0.0,
  'prem_missing_frac': 0.0,
  'funding_missing_frac': 0.0,
  'min_ts': '2025-01-01 00:00:00+00:00',
  'max_ts': '2025-01-31 23:45:00+00:00'},
 ('BTC-USDT', 2): {'ok': True,
  'out': 'C:\\Users\\quantbase\\Desktop\\marketdata\\norm\\master\\interval=15m\\year=2025\\month=02\\symbol=BTC-USDT\\part-2025-02.parquet',
  'rows': 2688,
  'mark_missing_frac': 0.0,
  'index_missing_frac': 0.0,
  'prem_missing_frac': 0.0,
  'funding_missing_frac': 0.0,
  'min_ts': '2025-02-01 00:00:00+00:00',
  'max_ts': '2025-02-28 23:45:00+00:00'},
 ('BTC-USDT', 3): {'ok': True,
  'out': 'C:\\Users\\quantbase\\Desktop\\marketdata\\norm\\master\\interval=15m\\year=2025\\month=03\\symbol=BTC-USDT\\part-2025-03.parquet',
  'rows': 2976,
  'mark_missing_frac': 0.0,
  'inde

In [6]:
# ------Join------

from pathlib import Path  # no installation needed
import calendar  # no installation needed
import pandas as pd  # already in env — no new install

DATA_ROOT = Path(r"C:\Users\quantbase\Desktop\marketdata")
INTERVAL = "15m"
YEAR = 2025

OUT_BASE = DATA_ROOT / "norm" / "master" / f"interval={INTERVAL}" / f"year={YEAR}"

# reports = {...}  # paste your dict here (or keep it from prior cell)

rep_rows = []
for (sym, mo), d in reports.items():
    row = {"symbol": sym, "month": mo, **d}
    rep_rows.append(row)

reports_df = pd.DataFrame(rep_rows).sort_values(["symbol", "month"]).reset_index(drop=True)
reports_df


,symbol,month,ok,out,rows,mark_missing_frac,index_missing_frac,prem_missing_frac,funding_missing_frac,min_ts,max_ts
0,ADA-USDT,1,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2976,0.0,0.0,0.0,0.0,2025-01-01 00:00:00+00:00,2025-01-31 23:45:00+00:00
1,ADA-USDT,2,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2688,0.0,0.0,0.0,0.0,2025-02-01 00:00:00+00:00,2025-02-28 23:45:00+00:00
2,ADA-USDT,3,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2976,0.0,0.0,0.0,0.0,2025-03-01 00:00:00+00:00,2025-03-31 23:45:00+00:00
3,ADA-USDT,4,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2880,0.0,0.0,0.0,0.0,2025-04-01 00:00:00+00:00,2025-04-30 23:45:00+00:00
4,ADA-USDT,5,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2976,0.0,0.0,0.0,0.0,2025-05-01 00:00:00+00:00,2025-05-31 23:45:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
79,XRP-USDT,8,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2976,0.0,0.0,0.0,0.0,2025-08-01 00:00:00+00:00,2025-08-31 23:45:00+00:00
80,XRP-USDT,9,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2880,0.0,0.0,0.0,0.0,2025-09-01 00:00:00+00:00,2025-09-30 23:45:00+00:00
81,XRP-USDT,10,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2976,0.0,0.0,0.0,0.0,2025-10-01 00:00:00+00:00,2025-10-31 23:45:00+00:00
82,XRP-USDT,11,True,C:\Users\quantbase\Desktop\marketdata\norm\mas...,2880,0.0,0.0,0.0,0.0,2025-11-01 00:00:00+00:00,2025-11-30 23:45:00+00:00


In [7]:
# rowcount and timestamp audit


def expected_rows_15m(year: int, month: int) -> int:
    days = calendar.monthrange(year, month)[1]
    return days * 24 * 4

audit_df = reports_df.copy()
audit_df["expected_rows"] = audit_df["month"].apply(lambda m: expected_rows_15m(YEAR, int(m)))
audit_df["rows_ok"] = audit_df["rows"] == audit_df["expected_rows"]
audit_df["all_missing_zero"] = (
    (audit_df["mark_missing_frac"] == 0.0)
    & (audit_df["index_missing_frac"] == 0.0)
    & (audit_df["prem_missing_frac"] == 0.0)
    & (audit_df["funding_missing_frac"] == 0.0)
)

# Any failures show up here
audit_df.loc[~(audit_df["rows_ok"] & audit_df["all_missing_zero"]), [
    "symbol","month","rows","expected_rows",
    "mark_missing_frac","index_missing_frac","prem_missing_frac","funding_missing_frac",
    "min_ts","max_ts","out"
]]


,symbol,month,rows,expected_rows,mark_missing_frac,index_missing_frac,prem_missing_frac,funding_missing_frac,min_ts,max_ts,out


In [8]:
# schema consistency check


# sample first 2 months per symbol
sample_paths = (
    reports_df.sort_values(["symbol","month"])
    .groupby("symbol")
    .head(2)["out"]
    .tolist()
)

schemas = []
for p in sample_paths:
    df = pd.read_parquet(p)
    schemas.append((p, tuple(df.columns), tuple(str(t) for t in df.dtypes)))

# show any column mismatches
col_sets = [set(cols) for _, cols, _ in schemas]
base = col_sets[0]
mismatched = [(p, sorted(list(set(cols) ^ base))) for (p, cols, _) in schemas if set(cols) != base]
mismatched[:10]


[]

In [9]:
# build YEAR “long” dataset from all symbol-month parts


def load_symbol_year_long(symbol: str) -> pd.DataFrame:
    files = sorted((OUT_BASE).glob(f"month=*/symbol={symbol}/part-*.parquet"))
    dfs = [pd.read_parquet(f) for f in files]
    df = pd.concat(dfs, ignore_index=True)
    df["symbol"] = symbol  # enforce
    df = df.sort_values("ts").reset_index(drop=True)
    return df

symbols = sorted(reports_df["symbol"].unique().tolist())

long_parts = []
for sym in symbols:
    long_parts.append(load_symbol_year_long(sym))

master_long = pd.concat(long_parts, ignore_index=True)

# key integrity
key_dupes = master_long.duplicated(subset=["symbol","ts"]).sum()
master_long = master_long.sort_values(["symbol","ts"]).reset_index(drop=True)

master_long.shape, key_dupes, master_long.head(3)


((245280, 14),
 np.int64(0),
                          ts      open_time  spot_close  mark_close  \
 0 2025-01-01 00:00:00+00:00  1735689600000      0.8512    0.850900   
 1 2025-01-01 00:15:00+00:00  1735690500000      0.8521    0.851900   
 2 2025-01-01 00:30:00+00:00  1735691400000      0.8541    0.853723   
 
    index_close  premium_close  basis_mark_vs_spot  basis_index_vs_spot  \
 0     0.850975       0.000000           -0.000352            -0.000264   
 1     0.852083      -0.000017           -0.000235            -0.000020   
 2     0.853985      -0.000084           -0.000441            -0.000135   
 
    funding_rate  funding_interval_hours     volume  quote_volume  trades  \
 0        0.0001                       8   839360.2  712349.52783    2397   
 1        0.0001                       8   496512.6  422710.98123    1745   
 2        0.0001                       8  1096408.0  936787.72716    2402   
 
      symbol  
 0  ADA-USDT  
 1  ADA-USDT  
 2  ADA-USDT  )

In [14]:
def qc_zero_nan(df: pd.DataFrame, col: str) -> dict:
    s = df[col]
    out = {
        "dtype": str(s.dtype),
        "rows": int(len(s)),
        "na_frac": float(s.isna().mean()),
        "zero_frac": float((s == 0).mean()) if pd.api.types.is_numeric_dtype(s) else None,
    }
    if pd.api.types.is_numeric_dtype(s):
        out |= {
            "min": float(s.min()),
            "p01": float(s.quantile(0.01)),
            "p50": float(s.quantile(0.50)),
            "p99": float(s.quantile(0.99)),
            "max": float(s.max()),
        }
    return out

# 1) check raw premium
print("RAW prem close qc:", qc_zero_nan(prem_m, "close"))

# 2) check raw funding
print("RAW funding rate qc:", qc_zero_nan(fund_m, "funding_rate"))

# 3) check master after join (your renamed columns)
print("MASTER premium qc:", qc_zero_nan(master, "premium_close"))
print("MASTER funding qc:", qc_zero_nan(master, "funding_rate"))


RAW prem close qc: {'dtype': 'float64', 'rows': 2976, 'na_frac': 0.0, 'zero_frac': 0.06283602150537634, 'min': -0.00181045, 'p01': -0.0011385375, 'p50': -0.0004325, 'p99': 8.71525e-05, 'max': 0.00098916}
RAW funding rate qc: {'dtype': 'float64', 'rows': 93, 'na_frac': 0.0, 'zero_frac': 0.0, 'min': -0.00019623, 'p01': -0.0001807556, 'p50': 4.491e-05, 'p99': 0.0001, 'max': 0.0001}
MASTER premium qc: {'dtype': 'float64', 'rows': 2976, 'na_frac': 0.0, 'zero_frac': 0.06283602150537634, 'min': -0.00181045, 'p01': -0.0011385375, 'p50': -0.0004325, 'p99': 8.71525e-05, 'max': 0.00098916}
MASTER funding qc: {'dtype': 'float64', 'rows': 2976, 'na_frac': 0.0, 'zero_frac': 0.0, 'min': -0.00019623, 'p01': -0.00019623, 'p50': 4.491e-05, 'p99': 0.0001, 'max': 0.0001}


In [16]:
# Proxy premium from mark vs index
master["prem_proxy_mark_vs_index"] = (master["mark_close"] / master["index_close"]) - 1.0

# Flag suspicious rows: premium exactly 0 while proxy is not close to 0
eps = 1e-6
mask = (master["premium_close"] == 0) & (master["prem_proxy_mark_vs_index"].abs() > 10*eps)

master.loc[mask, [
    "ts","spot_close","mark_close","index_close",
    "premium_close","prem_proxy_mark_vs_index",
    "basis_mark_vs_spot","basis_index_vs_spot"
]].head(5)


#If premium_close == 0 and prem_proxy_mark_vs_index ≈ 0: zero can be legitimate.
#If premium_close == 0 and prem_proxy_mark_vs_index is not near 0: it’s almost certainly a bad default/coercion.


,ts,spot_close,mark_close,index_close,premium_close,prem_proxy_mark_vs_index,basis_mark_vs_spot,basis_index_vs_spot
23,2025-12-01 05:45:00+00:00,12.09,12.092000,12.092713,0.0,-0.000059,0.000165,0.000224
70,2025-12-01 17:30:00+00:00,11.85,11.844693,11.847711,0.0,-0.000255,-0.000448,-0.000193
87,2025-12-01 21:45:00+00:00,12.03,12.025603,12.030131,0.0,-0.000376,-0.000365,0.000011
92,2025-12-01 23:00:00+00:00,12.07,12.074000,12.075265,0.0,-0.000105,0.000331,0.000436
96,2025-12-02 00:00:00+00:00,12.08,12.074000,12.075787,0.0,-0.000148,-0.000497,-0.000349


In [17]:
# save


OUT_LONG = DATA_ROOT / "norm" / "master_joined" / f"interval={INTERVAL}" / f"year={YEAR}"
OUT_LONG.mkdir(parents=True, exist_ok=True)
master_long.to_parquet(OUT_LONG / "master_long.parquet", index=False)
master_long.to_csv(OUT_LONG / "master_long.csv", index=False)


In [18]:
# build “wide” dataset (MultiIndex columns, clean + scalable)

# choose wide columns (add/remove)
WIDE_COLS = [
    "spot_close", "mark_close", "index_close", "premium_close",
    "funding_rate",
    "basis_mark_vs_spot", "basis_index_vs_spot",
    "volume", "quote_volume", "trades",
]

wide = (
    master_long[["ts","symbol"] + WIDE_COLS]
    .set_index(["ts","symbol"])
    .sort_index()
    .unstack("symbol")
)

wide.shape, wide.columns[:8]



((35040, 70),
 MultiIndex([('spot_close',  'ADA-USDT'),
             ('spot_close',  'BNB-USDT'),
             ('spot_close',  'BTC-USDT'),
             ('spot_close',  'ETH-USDT'),
             ('spot_close', 'LINK-USDT'),
             ('spot_close',  'SOL-USDT'),
             ('spot_close',  'XRP-USDT'),
             ('mark_close',  'ADA-USDT')],
            names=[None, 'symbol']))

In [19]:
wide_flat = wide.copy()
wide_flat.columns = [f"{feat}__{sym}" for feat, sym in wide_flat.columns]
wide_flat = wide_flat.sort_index()
wide_flat.shape, wide_flat.head(3)


((35040, 70),
                            spot_close__ADA-USDT  spot_close__BNB-USDT  \
 ts                                                                      
 2025-01-01 00:00:00+00:00                0.8512                704.01   
 2025-01-01 00:15:00+00:00                0.8521                704.63   
 2025-01-01 00:30:00+00:00                0.8541                706.21   
 
                            spot_close__BTC-USDT  spot_close__ETH-USDT  \
 ts                                                                      
 2025-01-01 00:00:00+00:00              93656.18               3348.50   
 2025-01-01 00:15:00+00:00              93761.90               3349.43   
 2025-01-01 00:30:00+00:00              93885.01               3356.09   
 
                            spot_close__LINK-USDT  spot_close__SOL-USDT  \
 ts                                                                       
 2025-01-01 00:00:00+00:00                  20.09                190.51   
 2025-01-01 00:15

In [20]:
wide_flat.to_parquet(OUT_LONG / "master_wide.parquet", index=True)
wide_flat.to_csv(OUT_LONG / "master_wide.csv", index=True)

In [21]:
# fast inspection of coverage on the joined outputs


inspect = {
    "long_rows": len(master_long),
    "unique_symbols": master_long["symbol"].nunique(),
    "min_ts": str(master_long["ts"].min()),
    "max_ts": str(master_long["ts"].max()),
    "dupes(symbol,ts)": int(master_long.duplicated(["symbol","ts"]).sum()),
    "missing_any_core": float(
        master_long[["spot_close","mark_close","index_close","premium_close","funding_rate"]].isna().any(axis=1).mean()
    ),
}
inspect


{'long_rows': 245280,
 'unique_symbols': 7,
 'min_ts': '2025-01-01 00:00:00+00:00',
 'max_ts': '2025-12-31 23:45:00+00:00',
 'dupes(symbol,ts)': 0,
 'missing_any_core': 0.0}

In [22]:
wide_flat.columns

Index(['spot_close__ADA-USDT', 'spot_close__BNB-USDT', 'spot_close__BTC-USDT',
       'spot_close__ETH-USDT', 'spot_close__LINK-USDT', 'spot_close__SOL-USDT',
       'spot_close__XRP-USDT', 'mark_close__ADA-USDT', 'mark_close__BNB-USDT',
       'mark_close__BTC-USDT', 'mark_close__ETH-USDT', 'mark_close__LINK-USDT',
       'mark_close__SOL-USDT', 'mark_close__XRP-USDT', 'index_close__ADA-USDT',
       'index_close__BNB-USDT', 'index_close__BTC-USDT',
       'index_close__ETH-USDT', 'index_close__LINK-USDT',
       'index_close__SOL-USDT', 'index_close__XRP-USDT',
       'premium_close__ADA-USDT', 'premium_close__BNB-USDT',
       'premium_close__BTC-USDT', 'premium_close__ETH-USDT',
       'premium_close__LINK-USDT', 'premium_close__SOL-USDT',
       'premium_close__XRP-USDT', 'funding_rate__ADA-USDT',
       'funding_rate__BNB-USDT', 'funding_rate__BTC-USDT',
       'funding_rate__ETH-USDT', 'funding_rate__LINK-USDT',
       'funding_rate__SOL-USDT', 'funding_rate__XRP-USDT',
       '

In [23]:
#-----Join BVOL-----

DATA_ROOT = Path(r"C:\Users\quantbase\Desktop\marketdata")
BVOL_DIR = DATA_ROOT / "raw" / "binance" / "bvol_index"

bvol_files = sorted(BVOL_DIR.rglob("part-*.parquet"))
print("bvol files:", len(bvol_files))
print("sample:", bvol_files[:3])

bvol_raw = pd.concat((pd.read_parquet(p) for p in bvol_files), ignore_index=True)
bvol_raw

bvol files: 1808
sample: [WindowsPath('C:/Users/quantbase/Desktop/marketdata/raw/binance/bvol_index/symbol=BTCBVOLUSDT/part-2023-06-20.parquet'), WindowsPath('C:/Users/quantbase/Desktop/marketdata/raw/binance/bvol_index/symbol=BTCBVOLUSDT/part-2023-06-21.parquet'), WindowsPath('C:/Users/quantbase/Desktop/marketdata/raw/binance/bvol_index/symbol=BTCBVOLUSDT/part-2023-06-22.parquet')]


,calc_time,symbol,base_asset,quote_asset,index_value,file_date,venue,dataset
0,1687219900000,BTCBVOLUSDT,BTCBVOL,USDT,47.3320,2023-06-20,binance,bvol_index
1,1687219901000,BTCBVOLUSDT,BTCBVOL,USDT,47.3321,2023-06-20,binance,bvol_index
2,1687219902000,BTCBVOLUSDT,BTCBVOL,USDT,47.3322,2023-06-20,binance,bvol_index
3,1687219903000,BTCBVOLUSDT,BTCBVOL,USDT,47.3322,2023-06-20,binance,bvol_index
4,1687219904000,BTCBVOLUSDT,BTCBVOL,USDT,47.3323,2023-06-20,binance,bvol_index
...,...,...,...,...,...,...,...,...
155823796,1765692354000,ETHBVOLUSDT,ETHBVOL,USDT,69.6490,2025-12-14,binance,bvol_index
155823797,1765692355006,ETHBVOLUSDT,ETHBVOL,USDT,69.6124,2025-12-14,binance,bvol_index
155823798,1765692356000,ETHBVOLUSDT,ETHBVOL,USDT,69.5760,2025-12-14,binance,bvol_index
155823799,1765692357001,ETHBVOLUSDT,ETHBVOL,USDT,69.5400,2025-12-14,binance,bvol_index


In [24]:
# normalise timestamp column to `ts` ---

# bvol_raw already loaded
bvol_raw = bvol_raw.copy()

# 1) timestamp
bvol_raw["ts"] = pd.to_datetime(bvol_raw["calc_time"], unit="ms", utc=True)

# 2) series id: BTCBVOL / ETHBVOL (use base_asset; it's canonical here)
bvol_raw["bvol_symbol"] = bvol_raw["base_asset"].astype(str)

# 3) value
bvol_raw["bvol_close"] = pd.to_numeric(bvol_raw["index_value"], errors="coerce")

# 4) keep minimal columns
bvol = (
    bvol_raw[["ts", "bvol_symbol", "bvol_close"]]
    .dropna(subset=["ts", "bvol_close"])
    .sort_values(["bvol_symbol", "ts"])
)

bvol.head(), bvol["bvol_symbol"].value_counts().head(10)


(                         ts bvol_symbol  bvol_close
 0 2023-06-20 00:11:40+00:00     BTCBVOL     47.3320
 1 2023-06-20 00:11:41+00:00     BTCBVOL     47.3321
 2 2023-06-20 00:11:42+00:00     BTCBVOL     47.3322
 3 2023-06-20 00:11:43+00:00     BTCBVOL     47.3322
 4 2023-06-20 00:11:44+00:00     BTCBVOL     47.3323,
 bvol_symbol
 BTCBVOL    77960681
 ETHBVOL    77863120
 Name: count, dtype: int64)

In [21]:
# resample BVOL to your 15m grid

# pick the same interval you used for master
BVOL_RULE = "15min"

bvol_15m = (
    bvol.set_index("ts")
        .groupby("bvol_symbol")["bvol_close"]
        .resample(BVOL_RULE)
        .last()
        .reset_index()
        .sort_values(["bvol_symbol", "ts"])
)

bvol_15m.head(), bvol_15m.tail()


(  bvol_symbol                        ts  bvol_close
 0     BTCBVOL 2023-06-20 00:00:00+00:00     47.3304
 1     BTCBVOL 2023-06-20 00:15:00+00:00     47.4568
 2     BTCBVOL 2023-06-20 00:30:00+00:00     47.5522
 3     BTCBVOL 2023-06-20 00:45:00+00:00     47.5671
 4     BTCBVOL 2023-06-20 01:00:00+00:00     47.5120,
        bvol_symbol                        ts  bvol_close
 174381     ETHBVOL 2025-12-14 05:00:00+00:00     69.8531
 174382     ETHBVOL 2025-12-14 05:15:00+00:00     69.8291
 174383     ETHBVOL 2025-12-14 05:30:00+00:00     69.8401
 174384     ETHBVOL 2025-12-14 05:45:00+00:00     69.8518
 174385     ETHBVOL 2025-12-14 06:00:00+00:00     69.5041)

In [22]:
# align BVOL to your canonical ts grid (15m) and create “wide” BVOL columns


# requires: wide_flat exists, index = ts (DatetimeIndex, UTC)
bvol_wide = (
    bvol_15m.pivot(index="ts", columns="bvol_symbol", values="bvol_close")
    .sort_index()
)

# align to master grid and ffill
bvol_wide = bvol_wide.reindex(wide_flat.index, method="ffill")

# rename columns to match your naming convention
bvol_wide.columns = [f"bvol_close__{c}" for c in bvol_wide.columns]

wide_flat_with_bvol = wide_flat.join(bvol_wide, how="left")

{c: float(wide_flat_with_bvol[c].isna().mean()) for c in bvol_wide.columns}



{'bvol_close__BTCBVOL': 5.7077625570776254e-05,
 'bvol_close__ETHBVOL': 0.0004280821917808219}

In [23]:
# Long format

# requires: master_long exists with columns ["symbol","ts", ...]
bvol_long = bvol_wide.reset_index().rename(columns={"index": "ts"})
master_long_with_bvol = master_long.merge(bvol_long, on="ts", how="left")

int(master_long_with_bvol.duplicated(["symbol","ts"]).sum()), master_long_with_bvol.shape

(0, (245280, 16))

In [24]:
master_long_with_bvol.to_csv(OUT_LONG / "bvol_long.csv", index=False)
wide_flat_with_bvol.to_csv(OUT_LONG / "master_wide_with_bvol.csv", index=True)

In [ ]:
#-------interpolation check (misc_)------

def grid_qc(df: pd.DataFrame, name: str):
    x = df.sort_values("open_time").copy()
    dt = x["open_time"].diff().dropna()
    return {
        "name": name,
        "rows": len(x),
        "interval_col_unique": sorted(x["interval"].unique()) if "interval" in x.columns else None,
        "min_open_time": int(x["open_time"].min()),
        "max_open_time": int(x["open_time"].max()),
        "step_ms_mode": int(dt.mode().iloc[0]) if len(dt) else None,
        "bad_step_frac": float((dt != dt.mode().iloc[0]).mean()) if len(dt) else 0.0,
        "open_time_mod_15m_bad_frac": float((x["open_time"] % 900_000 != 0).mean()),
    }

# Expect step_ms_mode == 900_000 and interval_col_unique includes "15m" for mark/index/prem datasets
grid_qc(mark_m, "mark")
grid_qc(index_m, "index")
grid_qc(prem_m, "premium")

{'name': 'premium',
 'rows': 2976,
 'interval_col_unique': ['15m'],
 'min_open_time': 1764547200000,
 'max_open_time': 1767224700000,
 'step_ms_mode': 900000,
 'bad_step_frac': 0.0,
 'open_time_mod_15m_bad_frac': 0.0}

In [29]:
# Intra-hour movement must exist in the RAW 15m mark/index/prem (not manufactured in master)
mark_intra = mark_m.assign(h=mark_m["ts"].dt.floor("h")).groupby("h")["close"].nunique().describe()
index_intra = index_m.assign(h=index_m["ts"].dt.floor("h")).groupby("h")["close"].nunique().describe()
prem_intra = prem_m.assign(h=prem_m["ts"].dt.floor("h")).groupby("h")["close"].nunique().describe()
mark_intra, index_intra, prem_intra

(count    744.000000
 mean       3.975806
 std        0.153753
 min        3.000000
 25%        4.000000
 50%        4.000000
 75%        4.000000
 max        4.000000
 Name: close, dtype: float64,
 count    744.0
 mean       4.0
 std        0.0
 min        4.0
 25%        4.0
 50%        4.0
 75%        4.0
 max        4.0
 Name: close, dtype: float64,
 count    744.000000
 mean       3.958333
 std        0.231178
 min        2.000000
 25%        4.000000
 50%        4.000000
 75%        4.000000
 max        4.000000
 Name: close, dtype: float64)